## 1 - Gerekli Kütüphanelerin Yüklenmesi

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

## 2 - Veriye Bakış

In [ ]:
train_final = pd.read_parquet('/kaggle/input/turkiye-is-bankasi-ml-challenge-5/train_final.parquet')
df = train_final.copy()
df.head()

## 3 - Veri Setinin Yapısal Bilgileri 

In [ ]:
df.info()

- feature_48 ve feature_49 sütunları diğer feature sütunların aksine tam sayı değerleri almaktadır. Bu sütunlar müşterilere has öznitelikler olduğundan dolayı feature_48 ve feature_49 sütunları ay gibi gün gibi tam değerlerden oluşmuş olabilir. Mesela uygulamayı kullandığı gün sayısı veya menülere tıklama sayısı gibi değerler olabilir

In [ ]:
df.describe().T

- n_seconds_1'in minimum değeri ile maksimum değeri arasında çok fark olmasına rağmen ortalaması ile standart sapması arasında çok az fark vardır. Bu demektir ki değişkenlerinin değerleri ortalamaya yakın bir dağılım göstermektedir. Minimum değer ile maksimum değer arasındaki bu kadar büyük farkı tetikleyen bir değer olduğunu tahmin etmekteyiz.
- n_seconds_2 ve n_seconds_3 içinde benzer şeyleri söyleyebiliriz.

## 4 - Kategorik Değişken Özellikleri

In [ ]:
kat_df = df.select_dtypes("object")
kat_df.head()

In [ ]:
# Telefon markalarının sınıflarına erişmek
kat_df.devicebrand.unique()

In [ ]:
# Telefon markalarının sınıflarının frekansına ulaşmak
kat_df["devicebrand"].value_counts()

- Anlaşılacağı gibi en çok kullanılan telefon markası "Apple" olarak geçmektedir.
- En çok kullanılan üç markanın ikisi Android cihaz üreticisi iken diğer ise IOS cihaz üreticisidir.
- Uygulamayı kullanan cihazların çoğunluğu Android cihazdır. 

In [ ]:
for i in kat_df["devicebrand"].value_counts().items():
    print(i)

In [ ]:
kat_df_cihaz = ["Apple", "Samsung", "Redmi", "Huawei", "Xiaomi"]

devicebrand_values = df["devicebrand"].value_counts().values
array_5 = np.array(devicebrand_values[0:5])

plt.title("En çok Kullanılan 5 Cihaz")
plt.pie(array_5,labels=kat_df_cihaz, autopct="%1.1f%%");

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(kat_df_cihaz, array_5)

plt.title("En Çok Kullanılan 5 Cihaz Markasının Frekansı")
plt.xlabel("Cihaz Markası")
plt.ylabel("Frekans")

plt.show()

In [ ]:
# Telefon operatörlerinin sınıflarına erişmek
kat_df.carrier.unique()

- Sınıfları dikkatli inceledğimizde 'GOKHAN TEPE' gibi telefon operatörü olmadığı halde isim soyisim gözüküyor. Burada ayrıca yabancı ülkelere ait telefon operatörleri de yer almaktadır.

In [ ]:
# Telefon operatörlerinin sınıflarının frekansına ulaşmak
kat_df["carrier"].value_counts()

- En çok kullanılan operatör Turkcell başta olmak sırasıyla Vodafone ve Türk Telekom gelmektedir.

In [ ]:
kat_df_carrier = ["Turkcell", "Vodafone TR", "Türk Telekom", "Bimcell", "Unkown"]


liste = list(df["carrier"].value_counts().values)
sirali_liste = sorted(liste, reverse=True)
array_5 = np.array(sirali_liste[0:5])


plt.title("En çok Kullanılan 5 Operatör")
plt.pie(array_5,labels=kat_df_carrier, autopct="%1.1f%%");

## 5 - Sürekli Değişken Özellikleri

In [ ]:
df_say = df.select_dtypes(include = ["float64","int64"])
df_say.head()

In [ ]:
df_say.hist(bins=100, figsize = (50,50));

- Değişkenlerin bazıları sağa ve sola çarpıklık göstermektedir ve curtosis değerlerini göz ardı ederek görsele dayanarak sivrilik olduğunu söyleyebiliriz.
- Tekrar görsele bakarak normal dağılıma benzer bir dağılım sergilediklerini söyleyebiliriz ancak istatistiki bir test yapmadan yanıltıcı olabilmektedir.

In [ ]:
# Sayısal değişkenler arasındaki korelasyona bakalım
df_say.corr()

In [ ]:
df_say.corr()[abs(df_say.corr()) > 0.5].stack().sort_values(ascending=True).to_frame()

In [ ]:
# Korelasyonu 0.5 ile 0.9 arasında olanları aldık
corr_matrix = df_say.corr()
sonuc = corr_matrix[((corr_matrix > 0.5) & (corr_matrix < 0.9)) | ((corr_matrix < -0.5) & (corr_matrix > -0.9))]
sonuc

- Sayısal değişkenler arasındaki korelasyonlar incelenerek orta şiddetli ve yüksek şiddetli korelasyona sahip olan değerler alınarak model kurma kısmında bu değişkenler ve dağılımları göz önünde bulundurulacaktır.